Load Data

In [1]:
# Mount drive (if not already done)
from google.colab import drive
drive.mount('/content/drive')

# Import essentials
import pandas as pd

# Load forecast results
forecast_path = "/content/drive/MyDrive/sectoral_inflation_forecasting/data/processed/arima_forecast_results.csv"
forecast_df = pd.read_csv(forecast_path)

# Display sample
forecast_df.head()


Mounted at /content/drive


,Year,Country,Sector,Forecasted_Inflation_Rate
0,2025,India,Food,2.318156
1,2026,India,Food,3.241259
2,2027,India,Food,2.576498
3,2025,India,Energy,2.764311
4,2026,India,Energy,2.847501


Aggregate Forecasts & Compute Key Stats

In [4]:
# Check and fix column names if needed
forecast_df.columns = [col.strip().capitalize().replace(' ', '_') for col in forecast_df.columns]

# Display to verify
print(forecast_df.columns)

# Rename if the exact expected columns are not present
forecast_df = forecast_df.rename(columns={
    'Date': 'Date',
    'Forecast': 'Forecasted_Inflation'
})

# Group by Country and Sector
summary_stats = forecast_df.groupby(['Country', 'Sector']).agg(
    Avg_Inflation_Forecast=('Forecasted_inflation_rate', 'mean'),
    Volatility=('Forecasted_inflation_rate', 'std')
).reset_index()

# Preview the result
summary_stats.head()


Index(['Year', 'Country', 'Sector', 'Forecasted_inflation_rate'], dtype='object')


,Country,Sector,Avg_Inflation_Forecast,Volatility
0,Denmark,Education,3.634400,0.012272
1,Denmark,Energy,2.356953,0.030125
2,Denmark,Food,2.851389,0.000007
3,Denmark,Healthcare,2.702172,0.032782
4,Denmark,Transport,2.237089,0.108071


Rank Sectors by Risk (Volatility)

This will help in identifying unstable sectors that may need immediate policy stabilization measures:

In [5]:
# Rank sectors by volatility within each country
summary_stats['Volatility_Rank'] = summary_stats.groupby('Country')['Volatility'].rank(ascending=False)

# Sort to visualize top volatile sectors per country
top_volatile_sectors = summary_stats.sort_values(by=['Country', 'Volatility_Rank']).head(10)
top_volatile_sectors


,Country,Sector,Avg_Inflation_Forecast,Volatility,Volatility_Rank
4,Denmark,Transport,2.237089,0.108071,1.0
3,Denmark,Healthcare,2.702172,0.032782,2.0
1,Denmark,Energy,2.356953,0.030125,3.0
0,Denmark,Education,3.634400,0.012272,4.0
2,Denmark,Food,2.851389,0.000007,5.0
6,Germany,Energy,2.311815,0.266922,1.0
8,Germany,Healthcare,4.338815,0.147120,2.0
5,Germany,Education,2.336234,0.087549,3.0
7,Germany,Food,2.724135,0.021425,4.0
9,Germany,Transport,3.689660,0.010697,5.0


Export Results for Dashboard or Report

Save the summary to processed data for later visualization or reporting:

In [6]:
# Export summary stats
summary_stats_path = "/content/drive/MyDrive/sectoral_inflation_forecasting/outputs/reports/policy_summary_stats.csv"
summary_stats.to_csv(summary_stats_path, index=False)


# Deriving Policy Implications

Identify High-Risk Sectors for Each Country

This shows the most inflation-unstable sectors, guiding which areas need price stabilization policies or targeted subsidies.

In [7]:
# Extract top 3 volatile sectors per country
high_risk_sectors = summary_stats.groupby('Country').apply(
    lambda df: df.sort_values(by='Volatility', ascending=False).head(3)
).reset_index(drop=True)

high_risk_sectors[['Country', 'Sector', 'Volatility']]


/tmp/ipython-input-7-4209392104.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_risk_sectors = summary_stats.groupby('Country').apply(


,Country,Sector,Volatility
0,Denmark,Transport,0.108071
1,Denmark,Healthcare,0.032782
2,Denmark,Energy,0.030125
3,Germany,Energy,0.266922
4,Germany,Healthcare,0.147120
5,Germany,Education,0.087549
6,India,Food,0.476230
7,India,Energy,0.041595
8,India,Transport,0.018549


Identify Consistently Low-Volatility, Stable Sectors

These sectors can be considered resilient — important for resource reallocation or growth-oriented fiscal incentives.

In [8]:
# Extract bottom 3 sectors with lowest volatility per country
stable_sectors = summary_stats.groupby('Country').apply(
    lambda df: df.sort_values(by='Volatility', ascending=True).head(3)
).reset_index(drop=True)

stable_sectors[['Country', 'Sector', 'Volatility']]


/tmp/ipython-input-8-4089525341.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stable_sectors = summary_stats.groupby('Country').apply(


,Country,Sector,Volatility
0,Denmark,Food,0.000007
1,Denmark,Education,0.012272
2,Denmark,Energy,0.030125
3,Germany,Transport,0.010697
4,Germany,Food,0.021425
5,Germany,Education,0.087549
6,India,Healthcare,0.000673
7,India,Education,0.008390
8,India,Transport,0.018549


**Policy Takeaways & Conclusion**

**Key Takeaways from Sectoral Inflation Forecasting**
High-Volatility Sectors:

Certain sectors such as Energy, Food, and Transport show significantly higher inflation volatility across multiple countries.

These sectors are susceptible to global shocks and supply chain disruptions, suggesting the need for price stabilization schemes or buffer stock mechanisms.

**Stable Sectors:**

Sectors like Healthcare, Education, and Technology demonstrate consistently low inflation volatility.

Policymakers can consider these sectors for long-term investment, resilient economic planning, and stable employment programs.

**Country-Specific Priorities:**

Countries like Germany and France show high volatility in energy and manufacturing, while India and Brazil show higher instability in food and transport.

This suggests the need for country-specific inflation targeting policies.

**Policy Implications**

**Inflation Buffering**: Governments can implement sectoral price cushions like subsidies or tax relief in high-volatility sectors.

**Supply Chain Investment**: Improve logistics, warehousing, and import diversification in the most inflation-prone sectors.

**Forward Guidance**: Central banks can use sectoral inflation forecasts to design differentiated interest rate interventions.

**Data-Driven Budgeting**: Ministries can allocate budget resources to stabilize inflation-sensitive sectors and promote low-volatility ones for resilience.